# Метрики качества классификации

В задачах классификации может быть много особенностей, влияющих на подсчет качества: различные цены ошибок, несбалансированность классов и т.д. Из-за этого существует большое количество метрик качества — каждая из них рассчитана на определенное сочетание свойств задачи и требований к ее решению.

Меры качества классификации можно разбить на две большие группы: предназначенные для алгоритмов, выдающих номера классов, и для алгоритмов, выдающих оценки принадлежности к классам. К первой группе относятся доля правильных ответов, точность, полнота, F-мера. Ко второй — площади под ROC- или PR-кривой.

1. Загрузите файл classification.csv. В нем записаны истинные классы объектов выборки (колонка true) и ответы некоторого классификатора (колонка pred).

In [1]:
import numpy as np
import pandas as pd

In [2]:
classification_data = pd.read_csv('_8b9c6d9ae39e206610c6fd96894615a5_classification.csv')
classification_data

,true,pred
0,1,0
1,1,1
2,1,1
3,0,0
4,1,1
...,...,...
195,0,0
196,0,0
197,1,0
198,0,1


2. Заполните таблицу ошибок классификации:
    Для этого подсчитайте величины TP, FP, FN и TN согласно их определениям. Например, FP — это количество объектов, имеющих класс 0, но отнесенных алгоритмом к классу 1. Ответ в данном вопросе — четыре числа через пробел.

In [3]:
tp = classification_data.query('true == 1 & pred == 1').count()
fp = classification_data.query('true == 0 & pred == 1').count()
fn = classification_data.query('true == 1 & pred == 0').count()
tn = classification_data.query('true == 0 & pred == 0').count()

print(f'TP: {tp}\nFP: {fp}\nFN: {fn}\nTN: {tn}')

TP: true    43
pred    43
dtype: int64
FP: true    34
pred    34
dtype: int64
FN: true    59
pred    59
dtype: int64
TN: true    64
pred    64
dtype: int64


3. Посчитайте основные метрики качества классификатора:

Accuracy (доля верно угаданных) — sklearn.metrics.accuracy_score
Precision (точность) — sklearn.metrics.precision_score
Recall (полнота) — sklearn.metrics.recall_score
F-мера — sklearn.metrics.f1_score

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [5]:
accuracy_score(classification_data.true, classification_data.pred)

0.535

In [6]:
precision_score(classification_data.true, classification_data.pred)

0.5584415584415584

In [7]:
recall_score(classification_data.true, classification_data.pred)

0.4215686274509804

In [8]:
f1_score(classification_data.true, classification_data.pred)

0.48044692737430167

4. Имеется четыре обученных классификатора. В файле scores.csv записаны истинные классы и значения степени принадлежности положительному классу для каждого классификатора на некоторой выборке:

для логистической регрессии — вероятность положительного класса (колонка score_logreg),

для SVM — отступ от разделяющей поверхности (колонка score_svm),

для метрического алгоритма — взвешенная сумма классов соседей (колонка score_knn),

для решающего дерева — доля положительных объектов в листе (колонка score_tree).

In [9]:
scores_data = pd.read_csv('_eee1b9e8188f61bc35d954fbeb94e325_scores.csv')
scores_data

,true,score_logreg,score_svm,score_knn,score_tree
0,0,0.683832,0.145976,0.787063,0.500000
1,1,0.801966,0.239511,1.000000,0.833333
2,0,0.382315,-0.245701,0.000000,0.000000
3,1,0.506797,-0.137058,0.000000,0.105263
4,1,0.488781,-0.154148,0.000000,0.105263
...,...,...,...,...,...
195,0,0.573801,-0.088203,0.284192,0.400000
196,0,0.624422,-0.012315,0.205437,0.400000
197,1,0.425538,-0.135673,0.382351,0.700000
198,0,0.905270,0.583806,1.000000,1.000000


5. Посчитайте площадь под ROC-кривой для каждого классификатора. Какой классификатор имеет наибольшее значение метрики AUC-ROC (укажите название столбца)? Воспользуйтесь функцией sklearn.metrics.roc_auc_score.

In [10]:
from sklearn.metrics import roc_auc_score

In [11]:
roc_auc_score(scores_data.true, scores_data.score_logreg)

0.719187675070028

In [12]:
roc_auc_score(scores_data.true, scores_data.score_svm)

0.7086834733893557

In [13]:
roc_auc_score(scores_data.true, scores_data.score_knn)

0.6351540616246498

In [14]:
roc_auc_score(scores_data.true, scores_data.score_tree)

0.6919267707082833

6. Какой классификатор достигает наибольшей точности (Precision) при полноте (Recall) не менее 70% ?

Чтобы получить ответ на этот вопрос, найдите все точки precision-recall-кривой с помощью функции sklearn.metrics.precision_recall_curve. Она возвращает три массива: precision, recall, thresholds. В них записаны точность и полнота при определенных порогах, указанных в массиве thresholds. Найдите максимальной значение точности среди тех записей, для которых полнота не меньше, чем 0.7.



In [15]:
from sklearn.metrics import precision_recall_curve

In [16]:
max_precision = 0
for classifier in [scores_data.score_logreg, scores_data.score_svm, 
                   scores_data.score_knn , scores_data.score_tree]:
    prc = precision_recall_curve(scores_data.true, classifier)
    prc_df = pd.DataFrame({'precision': prc[0], 
                          'recall': prc[1]})
    print(f'Classifier: {classifier.name} maxs with recall >= 0.7:\n{prc_df.query("recall >= 0.7 ").max()}\n')

Classifier: score_logreg maxs with recall >= 0.7:
precision    0.630252
recall       1.000000
dtype: float64

Classifier: score_svm maxs with recall >= 0.7:
precision    0.622807
recall       1.000000
dtype: float64

Classifier: score_knn maxs with recall >= 0.7:
precision    0.606557
recall       1.000000
dtype: float64

Classifier: score_tree maxs with recall >= 0.7:
precision    0.651786
recall       1.000000
dtype: float64

